In [ ]:
# 캘리용 동영상 파일 녹화

In [1]:
import cv2

In [2]:
# 설정
OUTPUT_FILENAME = "./result/stereo_calibration_video.avi"
CAMERA_INDEX = 0
FRAME_WIDTH = 1280
FRAME_HEIGHT = 480

In [3]:
# 카메라 init
cap = cv2.VideoCapture(CAMERA_INDEX)

# 해상도
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

# 실제 적용된 해상도와 FPS 가져오기 (OpenCV 내부 함수 사용)
codec = str(cap.get(cv2.CAP_PROP_FOURCC))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 혹시 카메라가 FPS 정보를 제대로 안 주면 기본값 30으로 설정
if fps == 0:
    fps = 30.0

print(f"카메라 설정: {codec} {width}x{height} @ {fps} FPS")

# 코덱
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = None
is_recording = False

print(f"--- 녹화 준비 완료 ---")
print(f"'r' 키: 녹화 시작/중지")
print(f"'q' 키: 종료")

try:
    while True:
        # 1. 프레임 읽기 (여기서 하드웨어 속도에 맞춰 자동으로 대기함)
        ret, frame = cap.read()
        if not ret:
            print("프레임 수신 실패")
            break

        display_frame = frame.copy()

        # 2. 녹화 로직
        if is_recording:
            if out is None:
                # 녹화 시작 시점에 VideoWriter 생성 (카메라의 실제 FPS 적용)
                out = cv2.VideoWriter(OUTPUT_FILENAME, fourcc, fps, (width, height))
                print(f"녹화 시작... (설정 FPS: {fps})")
            
            out.write(frame)
            
            # REC 표시
            cv2.circle(display_frame, (50, 50), 20, (0, 0, 255), -1)
            cv2.putText(display_frame, "REC", (80, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # 화면 출력 (절반 크기로 줄여서 보기)
        # preview = cv2.resize(display_frame, (width // 2, height // 2))
        cv2.imshow('Stereo Recorder', display_frame)

        # 3. 키 입력 대기 (OpenCV 내부 타이머 역할)
        # 1ms만 대기하며 키 입력을 확인합니다. 루프 속도는 cap.read()가 결정합니다.
        key = cv2.waitKey(1) & 0xFF

        if key == ord('r'):
            is_recording = not is_recording
            if not is_recording:
                print("녹화 중지 및 저장 완료.")
                if out is not None:
                    out.release()
                    out = None

        elif key == ord('q'):
            break

except Exception as e:
    print(e)

finally:
    # 자원 반환
    if out is not None:
        out.release()
    cap.release()
    cv2.destroyAllWindows()

카메라 설정: 1196444237.0 1280x480 @ 30.0 FPS
--- 녹화 준비 완료 ---
'r' 키: 녹화 시작/중지
'q' 키: 종료
